# 加速：多線程爬蟲



* 了解知乎 API 使用方式與回傳內容
* 撰寫程式存取 API 且添加標頭

## 作業目標

* 找一個之前實作過的爬蟲改用多線程改寫，比較前後時間的差異。
* 使用Day_016_Wikipedia_Ans.ipynb改寫


In [3]:
import requests
import re
from bs4 import BeautifulSoup
import os

## 先定義一個我們想搜尋的字詞，並將它轉換成UTF-8編碼後的URL

In [2]:
input_keyword = "王建民_(棒球員)"  # 這裡可以自己定義有興趣的關鍵字

utf8_url = repr(input_keyword.encode('UTF-8')).upper()  # 編碼成UTF-8並轉成大寫字元
utf8_url = utf8_url.replace("\\X", "%")                 # 用 '%' 取代 '\X' 
print("%s: %s" % (input_keyword, utf8_url[2:-1:1]))     # 擷取中間的編碼結果

# 組成Wiki關鍵字搜尋的網址格式
root_keyword_link = '/wiki/' + utf8_url[2:-1:1]
print(root_keyword_link)

王建民_(棒球員): %E7%8E%8B%E5%BB%BA%E6%B0%91_(%E6%A3%92%E7%90%83%E5%93%A1)
/wiki/%E7%8E%8B%E5%BB%BA%E6%B0%91_(%E6%A3%92%E7%90%83%E5%93%A1)


### 範例1：送出關鍵字請求後，爬取該關鍵字的文章內容

In [5]:
# 模擬封包的標頭
headers = {
    'authority': 'zh.wikipedia.org',
    'method': 'GET',
    'path': '/wiki/' + root_keyword_link,
    'scheme': 'https',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7,zh-CN;q=0.6',
    'cookie': 'GeoIP=TW:TPE:Taipei:25.05:121.53:v4; TBLkisOn=0; mwPhp7Seed=8b8; WMF-Last-Access-Global=04-Jun-2019; WMF-Last-Access=04-Jun-2019',
    'dnt': '1',
    #'if-modified-since': 'Tue, 04 Jun 2019 12:03:22 GMT',
    'referer': 'https://zh.wikipedia.org/wiki/Wikipedia:%E9%A6%96%E9%A1%B5',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36'
}    

url = 'https://zh.wikipedia.org' + root_keyword_link  # 組合關鍵字查詢URL
resp = requests.get(url, headers=headers)
resp.encoding = 'utf-8'

html = BeautifulSoup(resp.text, "lxml")
content = html.find(name='div', attrs={'id':'mw-content-text'}).find_all(name='p')

#
# 解析回傳資料，並萃取文章內容
#
for paragraph in content:
    print(paragraph.get_text())


 
美國職棒

獨立聯盟

教練時期
MLB

中華職棒（CPBL）

王建民（英語：Chien-Ming Wang，1980年3月31日－）是臺灣職業棒球運動員，位置為投手，臺南市關廟區人。曾效力於美國職棒大聯盟紐約洋基、華盛頓國民、多倫多藍鳥、堪薩斯市皇家等隊，在大聯盟9個球季累計68勝（34敗），是台灣投手在大聯盟中累計勝投最高者。許多臺灣媒體在報導王建民的新聞時，常暱稱其為「建仔」、「王葛格」、「臺灣之光」，美國媒體也稱之為「Star of Taiwan」、「Taiwan lightning」、「Ace of Silence」並稱王引起的風潮為「王建民現象」[1]。

王建民於1980年3月31日出生在台南縣郊區關廟鄉新埔村（今臺南市關廟區新埔里），之後主要於台南市成長。生父王炳穎（三叔）在家排行第三，小時候因為大哥王炳煌無子嗣，王炳穎便決定將王建民過繼至長兄戶口下，由其大伯養育長大[2]。其大伯經營工廠。 王建民在就讀崇學國小期間，因不喜歡唸書且身高過人而加入籃球校隊。在兼任籃球與棒球校隊教練的鼓勵下開始打棒球，於校隊徵選時自願加入棒球隊，在崇學國小少棒隊中擔任投手。

1991年時，王建民當選成為第九屆世界軟式少棒錦標賽中華代表隊球員，成為其第一次入選出國比賽的代表隊選手。就讀建興國中時，經由教練張錫傑調教後，在未來的發展上奠定了深厚的根基（建興國中在王建民之後也培養岀郭泓志、胡金龍、羅錦龍等職棒選手）。儘管養父母強烈反對他打棒球，希望他能夠成為企業家，但在王建民決心、毅力與堅持，最後還是讓他繼續打球。青棒時期的王建民選擇加入傳統青棒勁旅榮工隊，當時的榮工隊即中華中學的前身，總教練為蕭文勝。

在榮工隊時期，高中生的王建民逐步強化了自身的投球能力與體格，擁有達190公分的身高與增重後勻稱的身材，最快球速也進步到了143km/h左右，國外球探亦開始將王列入觀察名單。1997年，榮工青棒隊獲得全國棒球選拔賽冠軍，王建民也因此入選在加拿大舉行的第十七屆「IBAF世界青棒錦標賽」中華代表隊選手。由於需要辦理出國護照等證件，王建民便請父母將戶籍資料從家鄉臺南寄至球隊所在的臺北，但也因此發現戶籍謄本資料上登記自己為養子。無法接受事實的王建民連夜趕回台南詢問父母，才知道親生父母是自己的三叔。

在發現自己的三叔是親生父親、而從小扶養長大的父母則是養父母後，王建民一開始

### 範例2：從爬取的文章內容中，擷取出有外部連結的關鍵字。這些關鍵字在文章中是以藍色字體顯示，會連到外部的網頁，並解釋其內容。

In [6]:
for ext_link in content:
    a_tag = ext_link.find_all('a', href=re.compile("^(/wiki/)((?!;)\S)*$"))
    if len(a_tag) > 0:
        for link_string in a_tag:
            a_link = link_string["href"]       # 外部連結的網址
            a_keyword = link_string.get_text()  # 外部連結的中文名稱
            print("外部連結: [%s] %s" % (a_keyword, a_link))

外部連結: [臺灣] /wiki/%E8%87%BA%E7%81%A3
外部連結: [職業棒球] /wiki/%E8%81%B7%E6%A5%AD%E6%A3%92%E7%90%83
外部連結: [運動員] /wiki/%E9%81%8B%E5%8B%95%E5%93%A1
外部連結: [投手] /wiki/%E6%8A%95%E6%89%8B
外部連結: [臺南市] /wiki/%E8%87%BA%E5%8D%97%E5%B8%82
外部連結: [關廟區] /wiki/%E9%97%9C%E5%BB%9F%E5%8D%80
外部連結: [美國職棒大聯盟] /wiki/%E7%BE%8E%E5%9C%8B%E8%81%B7%E6%A3%92%E5%A4%A7%E8%81%AF%E7%9B%9F
外部連結: [紐約洋基] /wiki/%E7%B4%90%E7%B4%84%E6%B4%8B%E5%9F%BA
外部連結: [華盛頓國民] /wiki/%E8%8F%AF%E7%9B%9B%E9%A0%93%E5%9C%8B%E6%B0%91
外部連結: [堪薩斯市皇家] /wiki/%E5%A0%AA%E8%96%A9%E6%96%AF%E5%B8%82%E7%9A%87%E5%AE%B6
外部連結: [大聯盟] /wiki/%E5%A4%A7%E8%81%AF%E7%9B%9F
外部連結: [台灣] /wiki/%E5%8F%B0%E7%81%A3
外部連結: [投手] /wiki/%E6%8A%95%E6%89%8B
外部連結: [勝投] /wiki/%E8%83%9C%E6%8A%95
外部連結: [臺灣] /wiki/%E8%87%BA%E7%81%A3
外部連結: [臺灣之光] /wiki/%E8%87%BA%E7%81%A3%E4%B9%8B%E5%85%89
外部連結: [臺南市] /wiki/%E8%87%BA%E5%8D%97%E5%B8%82
外部連結: [關廟區] /wiki/%E9%97%9C%E5%BB%9F%E5%8D%80
外部連結: [過繼] /wiki/%E9%81%8E%E7%B9%BC
外部連結: [崇學國小] /wiki/%E8%87%BA%E5%8D%97%E5%B8%82%E6%9D%B1%E5%8D%80%E5%B4%87%E5

## 作業：接下來定義一個爬蟲函數，這個函數的主要工作為：
### (1) 爬取當前關鍵字的解釋，並存入檔案(因為文章內容太多會佔滿整個頁面，所以存程檔案，方便後續檢視)
### (2) 萃取出當前關鍵字所引用的外部連結，當作新的查詢關鍵字
### (3) 把第(2)擷取到的關鍵字當作新的關鍵字，回到第(1)步，爬取新的關鍵字解釋。

In [7]:
def WikiArticle(key_word_link, key_word, recursive):
    
    if (recursive <= max_recursive_depth):
        print("遞迴層[%d] - %s (%s)" % (recursive, key_word_link, key_word))
        
        # 模擬封包的標頭
        headers = {
            'authority': 'zh.wikipedia.org',
            'method': 'GET',
            'path': '/wiki/' + key_word_link,
            'scheme': 'https',
            'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
            'accept-encoding': 'gzip, deflate, br',
            'accept-language': 'zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7,zh-CN;q=0.6',
            'cookie': 'GeoIP=TW:TPE:Taipei:25.05:121.53:v4; TBLkisOn=0; mwPhp7Seed=8b8; WMF-Last-Access-Global=04-Jun-2019; WMF-Last-Access=04-Jun-2019',
            'dnt': '1',
            #'if-modified-since': 'Tue, 04 Jun 2019 12:03:22 GMT',
            'referer': 'https://zh.wikipedia.org/wiki/Wikipedia:%E9%A6%96%E9%A1%B5',
            'upgrade-insecure-requests': '1',
            'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36'
        }    

        url = 'https://zh.wikipedia.org' + key_word_link  # 組合關鍵字查詢URL
        resp = requests.get(url, headers=headers)
        resp.encoding = 'utf-8'

        html = BeautifulSoup(resp.text, "lxml")
        content = html.find(name='div', attrs={'id':'mw-content-text'}).find_all(name='p')
        
        #
        # Part 1: 請參考範例1，爬取當前關鍵字的文章內容。
        #         因為內容太多，我們把它寫入檔案，並以關鍵字作為檔案名稱，以便稍後查閱內容。
        #         請先建立一個名為"WikiArticle"的資料夾，爬取到的文章內容會放在這個資料夾底下。
        #
        dir_ = 'WikiArticle'
        if not os.path.exists(dir_):
            os.makedirs(dir_)
        
        with open('{}/{}.txt'.format(dir_, key_word.replace('/', '_')), 'w', encoding='utf8') as f_:
            for paragraph in content:
                f_.write(paragraph.get_text())
       
        #
        # Part 2: 請參考範例2，萃取出本篇文章中所延伸引用的外部連結，並儲存在external_link_dict
        #
        external_link_dict = dict({})

        for ext_link in content:
            a_tag = ext_link.find_all('a', href=re.compile("^(/wiki/)((?!;)\S)*$"))
            if len(a_tag) > 0:
                for link_string in a_tag:
                    a_link = link_string["href"]       # 外部連結的網址
                    a_keyword = link_string.get_text()  # 外部連結的中文名稱
                    #print("外部連結: %s [%s]" % (a_link, a_keyword))
                    external_link_dict[a_link] = a_keyword
                    
        #
        # Part 3: 將Part 2所收集的外部連結，當作新的關鍵字，繼續迭代深入爬蟲
        #
        if (len(external_link_dict) > 0):
            
            recursive = recursive + 1  # 遞迴深度加1
            
            for k, v in external_link_dict.items():
                WikiArticle(k, v, recursive)  # 再次呼叫同樣的函數，執行同樣的流程
                

### 執行前個步驟定義好的爬蟲主程式

In [8]:
# 定義爬取的遞迴深度。深度不要訂太深，否則會爬很久。
import time
max_recursive_depth = 1
begTime=time.time()
print('BEG_TIME=',time.time())
WikiArticle(root_keyword_link, input_keyword, 0)
endTime=time.time()
print('END_TIME=',time.time())
print('INTERVAL_TIME=',endTime-begTime)


BEG_TIME= 1579313966.932863
遞迴層[0] - /wiki/%E7%8E%8B%E5%BB%BA%E6%B0%91_(%E6%A3%92%E7%90%83%E5%93%A1) (王建民_(棒球員))
遞迴層[1] - /wiki/%E8%87%BA%E7%81%A3 (臺灣)
遞迴層[1] - /wiki/%E8%81%B7%E6%A5%AD%E6%A3%92%E7%90%83 (職業棒球)
遞迴層[1] - /wiki/%E9%81%8B%E5%8B%95%E5%93%A1 (運動員)
遞迴層[1] - /wiki/%E6%8A%95%E6%89%8B (投手)
遞迴層[1] - /wiki/%E8%87%BA%E5%8D%97%E5%B8%82 (臺南市)
遞迴層[1] - /wiki/%E9%97%9C%E5%BB%9F%E5%8D%80 (關廟區)
遞迴層[1] - /wiki/%E7%BE%8E%E5%9C%8B%E8%81%B7%E6%A3%92%E5%A4%A7%E8%81%AF%E7%9B%9F (美國職棒大聯盟)
遞迴層[1] - /wiki/%E7%B4%90%E7%B4%84%E6%B4%8B%E5%9F%BA (紐約洋基)
遞迴層[1] - /wiki/%E8%8F%AF%E7%9B%9B%E9%A0%93%E5%9C%8B%E6%B0%91 (華盛頓國民)
遞迴層[1] - /wiki/%E5%A0%AA%E8%96%A9%E6%96%AF%E5%B8%82%E7%9A%87%E5%AE%B6 (堪薩斯市皇家)
遞迴層[1] - /wiki/%E5%A4%A7%E8%81%AF%E7%9B%9F (大聯盟)
遞迴層[1] - /wiki/%E5%8F%B0%E7%81%A3 (台灣)
遞迴層[1] - /wiki/%E8%83%9C%E6%8A%95 (勝投)
遞迴層[1] - /wiki/%E8%87%BA%E7%81%A3%E4%B9%8B%E5%85%89 (臺灣之光)
遞迴層[1] - /wiki/%E9%81%8E%E7%B9%BC (過繼)
遞迴層[1] - /wiki/%E8%87%BA%E5%8D%97%E5%B8%82%E6%9D%B1%E5%8D%80%E5%B4%87%E5%AD%B8%E5%

In [11]:
# 加入多執行續，加快時間
# _thread.start_new_thread( print_time,("Thread-1", range(0, 5, 2), 2, ) )
import _thread
def myThread(key_word_link, key_word, recursive, multithread=False):
    _thread.start_new_thread( WikiArticle,(key_word_link, key_word, recursive, multithread))

In [12]:
import os
def WikiArticle(key_word_link, key_word, recursive,multithread=False):
    
    if (recursive <= max_recursive_depth):
        print("遞迴層[%d] - %s (%s)" % (recursive, key_word_link, key_word))
        
        # 模擬封包的標頭
        headers = {
            'authority': 'zh.wikipedia.org',
            'method': 'GET',
            'path': '/wiki/' + key_word_link,
            'scheme': 'https',
            'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
            'accept-encoding': 'gzip, deflate, br',
            'accept-language': 'zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7,zh-CN;q=0.6',
            'cookie': 'GeoIP=TW:TPE:Taipei:25.05:121.53:v4; TBLkisOn=0; mwPhp7Seed=8b8; WMF-Last-Access-Global=04-Jun-2019; WMF-Last-Access=04-Jun-2019',
            'dnt': '1',
            #'if-modified-since': 'Tue, 04 Jun 2019 12:03:22 GMT',
            'referer': 'https://zh.wikipedia.org/wiki/Wikipedia:%E9%A6%96%E9%A1%B5',
            'upgrade-insecure-requests': '1',
            'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36'
        }    

        url = 'https://zh.wikipedia.org' + key_word_link  # 組合關鍵字查詢URL
        resp = requests.get(url, headers=headers)
        resp.encoding = 'utf-8'

        html = BeautifulSoup(resp.text, "lxml")
        content = html.find(name='div', attrs={'id':'mw-content-text'}).find_all(name='p')
        
        #
        # Part 1: 請參考範例1，爬取當前關鍵字的文章內容。
        #         因為內容太多，我們把它寫入檔案，並以關鍵字作為檔案名稱，以便稍後查閱內容。
        #         請先建立一個名為"WikiArticle"的資料夾，爬取到的文章內容會放在這個資料夾底下。
        #
        dir_ = 'WikiArticle'
        if not os.path.exists(dir_):
            os.makedirs(dir_)
        
        with open('{}/{}.txt'.format(dir_, key_word.replace('/', '_')), 'w', encoding='utf8') as f_:
            for paragraph in content:
                f_.write(paragraph.get_text())
       
        #
        # Part 2: 請參考範例2，萃取出本篇文章中所延伸引用的外部連結，並儲存在external_link_dict
        #
        external_link_dict = dict({})

        for ext_link in content:
            a_tag = ext_link.find_all('a', href=re.compile("^(/wiki/)((?!;)\S)*$"))
            if len(a_tag) > 0:
                for link_string in a_tag:
                    a_link = link_string["href"]       # 外部連結的網址
                    a_keyword = link_string.get_text()  # 外部連結的中文名稱
                    #print("外部連結: %s [%s]" % (a_link, a_keyword))
                    external_link_dict[a_link] = a_keyword
                    
        #
        # Part 3: 將Part 2所收集的外部連結，當作新的關鍵字，繼續迭代深入爬蟲
        #
        if (len(external_link_dict) > 0):
            
            recursive = recursive + 1  # 遞迴深度加1

            thread_list = []
            for k, v in external_link_dict.items():
                if multithread:
                    thread = myThread(k, v, recursive,multithread)
                    #thread_list.append(thread)
                    #thread.start()
                else:
                    WikiArticle(k, v, recursive)  # 再次呼叫同樣的函數，執行同樣的流程

            if multithread:
                for thread in thread_list:
                    while thread_list[thread].locked(): pass


In [13]:
max_recursive_depth = 1
begTime=time.time()
print('BEG_TIME=',time.time())
WikiArticle(root_keyword_link, input_keyword, 0,multithread=True)

# 等待所有子執行緒結束
#while 1:
#   pass   
#print("Done.")
#endTime=time.time()
#print('END_TIME=',time.time())
#print('INTERVAL_TIME=',endTime-begTime)

BEG_TIME= 1579314165.6082268
遞迴層[0] - /wiki/%E7%8E%8B%E5%BB%BA%E6%B0%91_(%E6%A3%92%E7%90%83%E5%93%A1) (王建民_(棒球員))
遞迴層[1] - /wiki/%E9%97%9C%E5%BB%9F%E5%8D%80 (關廟區)遞迴層[1] - /wiki/%E9%83%AD%E6%B3%93%E5%BF%97 (郭泓志)遞迴層[1] - /wiki/%E7%B1%83%E7%90%83 (籃球)遞迴層[1] - /wiki/%E5%8F%B0%E5%8D%97%E5%B8%82%E7%AB%8B%E5%BB%BA%E8%88%88%E5%9C%8B%E6%B0%91%E4%B8%AD%E5%AD%B8 (建興國中)遞迴層[1] - /wiki/%E7%BE%85%E9%8C%A6%E9%BE%8D (羅錦龍)遞迴層[1] - /wiki/%E5%BB%BA%E8%88%88%E5%9C%8B%E4%B8%AD (建興國中)遞迴層[1] - /wiki/%E8%81%B7%E6%A5%AD%E6%A3%92%E7%90%83 (職業棒球)

遞迴層[1] - /wiki/%E5%8F%B0%E7%81%A3 (台灣)

遞迴層[1] - /wiki/%E5%A0%AA%E8%96%A9%E6%96%AF%E5%B8%82%E7%9A%87%E5%AE%B6 (堪薩斯市皇家)
遞迴層[1] - /wiki/%E6%97%A5%E6%9C%AC%E8%81%B7%E6%A3%92 (日本職棒)遞迴層[1] - /wiki/%E8%87%BA%E5%8C%97 (臺北)
遞迴層[1] - /wiki/%E4%B8%96%E7%95%8C%E9%9D%92%E6%A3%92%E9%8C%A6%E6%A8%99%E8%B3%BD (世界青棒錦標賽)
遞迴層[1] - /wiki/%E7%B4%90%E7%B4%84%E6%B4%8B%E5%9F%BA (紐約洋基)遞迴層[1] - /wiki/%E8%87%BA%E5%8D%97 (臺南)

遞迴層[1] - /wiki/%E8%83%9C%E6%8A%95 (勝投)遞迴層[1] - /wiki/%E9%81%8E%E7%B9%BC